In [1]:
import psycopg2
import requests
from datetime import datetime
import time
from bs4 import BeautifulSoup
import pandas as pd
import os
import codecs

In [2]:
connection = psycopg2.connect(
    database="postgres",
    user='postgres',
    password='Samara1!',
    host='localhost',
    port='5432'
)

cur = connection.cursor()

In [3]:
postgreSQL_select_Query = ("""
                           SELECT dg.link_url, dg.game_bk, dg.calendar_year, dg.calendar_month, dg.calendar_day 
                           FROM daily_games dg 
                           LEFT JOIN etl.box_score_capture_record bs ON bs.game_bk = dg.game_bk 
                           WHERE bs.game_bk IS NULL 
                           ORDER BY date_key DESC 
                           """)

In [4]:
cur.execute(postgreSQL_select_Query)
box_links = cur.fetchall()

In [ ]:
l = []
for row in box_links:
    x=[]
    URL = "https://www.baseball-reference.com"+row[0]
    page = requests.get(URL)
    path = "GameFiles/" + row[2] + "/" + row[3] + "/" + row[4]
    isExist = os.path.exists(path)
    if not isExist:
        os.makedirs(path)
    f = open("GameFiles/" + row[2] + "/" + row[3] + "/" + row[4] + "/" + row[1]+".html","w+")
    f.write(str(page.content))
    f.close()
    now = datetime.now()
    dt_string = now.strftime("%Y/%m/%d %H:%M:%S")
    x = (row[1],dt_string)
    cur.execute("INSERT into etl.box_score_capture_record(game_bk, created_at) VALUES (%s, %s)", x)
    connection.commit()
    time.sleep(3)
cur.close()
connection.close()